# Ollama PDF RAG Notebook

In [ ]:
# !pip install -r requirements.txt

## Import Libraries


In [ ]:
# Imports
from PyPDF2 import PdfReader

from langchain_ollama import OllamaEmbeddings

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Load PDF

In [ ]:
def get_pdf_text(pdf_path):
    text = ""
    pdf_reader = PdfReader(pdf_path)
    for page in pdf_reader.pages:
        text += page.extract_text()

    return text

In [ ]:
local_path = "Retrievel-Augmented-Generation-for-NLP.pdf"
data = get_pdf_text(local_path)

print("PDF loaded successfully.")

## Split text into chunks

In [ ]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
chunks = get_text_chunks(data)
print(f"Text split into {len(chunks)} chunks")

## Create vector database

In [ ]:
def get_vectordb(text_chunks):
    # embeddings = OpenAIEmbeddings()
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    vectordb = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectordb

In [ ]:
# !ollama pull nomic-embed-text

In [ ]:
vectordb = get_vectordb(chunks)

## Set up LLM and Retrieval

In [ ]:
# !ollama pull llama3


In [ ]:
# Set up LLM and retrieval
local_model = "llama3"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [ ]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vectordb.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [ ]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [ ]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [ ]:
# Example 1
chat_with_pdf("What is the main idea of this document?")

In [ ]:
chat_with_pdf(" Explain me the RAG-Sequence Model?")

## Clean up (optional)

In [ ]:
# # Optional: Clean up when done 
# vector_db.delete_collection()
# print("Vector database deleted successfully")